# **Problem Statement**


###  Use rawdata as input and derive
1. Datewise total duration for each inside and outside.
2. Datewise number of picking and placing activity done.

### Import Libraries

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import datetime

### Dataset Loading

In [2]:
# Load Dataset
raw_df= pd.read_csv("D:\\ResoluteAI Assignment\\data\\rawdata.csv")

### Dataset First View

In [3]:
# Dataset First Look
raw_df.head()

,date,time,sensor,location,number,activity,position,location.1
0,1/16/2024,10:10:30 AM,0.0,A1,1,placed,inside,A1
1,1/16/2024,10:12:30 AM,1.0,A2,1,picked,inside,A2
2,1/16/2024,10:28:30 AM,1.0,A3,1,picked,Inside,A3
3,1/16/2024,10:38:20 AM,0.0,A4,1,placed,Inside,A4
4,1/16/2024,10:45:20 AM,1.0,A5,1,placed,inside,A5


### Dataset Rows & Columns count

In [4]:
# Dataset Rows & Columns count
raw_df.shape

(175, 8)

### Dataset Information

In [5]:
# Dataset Info
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        175 non-null    object 
 1   time        175 non-null    object 
 2   sensor      174 non-null    float64
 3   location    175 non-null    object 
 4   number      175 non-null    int64  
 5   activity    175 non-null    object 
 6   position    175 non-null    object 
 7   location.1  175 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 11.1+ KB


#### Duplicate Values

In [6]:
# Dataset Duplicate Value Count
raw_df.duplicated().sum()

0

#### Missing Values/Null Values

In [7]:
# Missing Values/Null Values Count
raw_df.isnull().sum()

date          0
time          0
sensor        1
location      0
number        0
activity      0
position      0
location.1    0
dtype: int64

In [8]:
raw_df.loc[raw_df.isnull().any(axis=1)]

,date,time,sensor,location,number,activity,position,location.1
150,1/18/2024,7:10:00 PM,NaN,A31,1,placed,Inside,A31


the dataset contain 175 rows and 8 column having zero duplicates value but one null value in the "sensor" column.

## ***2. Understanding Your Variables***

In [9]:
# Dataset Columns
raw_df.columns

Index(['date', 'time', 'sensor', 'location', 'number', 'activity', 'position',
       'location.1'],
      dtype='object')

In [10]:
# Dataset Describe
raw_df.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
sensor,174.0,0.51,0.50,0.0,0.0,1.0,1.0,1.0
number,175.0,1.17,0.37,1.0,1.0,1.0,1.0,2.0


### Variables Description

### Check Unique Values for each variable.

In [11]:
# Check Unique Values for each variable.
raw_df.nunique()

date            3
time          140
sensor          2
location       34
number          2
activity        2
position        3
location.1     34
dtype: int64

In [12]:
print(f" Starting Time- {raw_df['time'].min()}")
print(f" Ending age- {raw_df['time'].max()}")

 Starting Time- 10:10:30 AM
 Ending age- 7:30:20 PM


In [13]:
raw_df.value_counts("date")

date
1/16/2024    80
1/18/2024    76
1/17/2024    19
Name: count, dtype: int64

In [14]:
raw_df["sensor"].value_counts(dropna=False)

sensor
1.0    89
0.0    85
NaN     1
Name: count, dtype: int64

In [15]:
raw_df.value_counts("number")

number
1    146
2     29
Name: count, dtype: int64

In [16]:
raw_df.value_counts("activity")

activity
placed    88
picked    87
Name: count, dtype: int64

In [17]:
raw_df.value_counts("position")

position
inside     156
Inside      15
outside      4
Name: count, dtype: int64

It is suspecious that the position column contain "inside" and "Inside" value, whose meaning is same but written in difference cases.

## 3. ***Data Wrangling***

### Data Wrangling Code

In [18]:
# Changing the datatype of date column

raw_df["date"]=pd.to_datetime(raw_df['date'])

In [19]:
# Changing the datatype of time column

raw_df["time"]=pd.to_datetime(raw_df['time'])

C:\Users\rajiv\AppData\Local\Temp\ipykernel_18048\689246453.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  raw_df["time"]=pd.to_datetime(raw_df['time'])


In [20]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        175 non-null    datetime64[ns]
 1   time        175 non-null    datetime64[ns]
 2   sensor      174 non-null    float64       
 3   location    175 non-null    object        
 4   number      175 non-null    int64         
 5   activity    175 non-null    object        
 6   position    175 non-null    object        
 7   location.1  175 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 11.1+ KB


In [21]:
# Replacing null Values

In [22]:
mode_value = raw_df["sensor"].mode()[0]  # Get the mode value
raw_df["sensor"] = raw_df["sensor"].fillna(mode_value)

We didnt drop the null values because the dataset contain less rows,so we are imputing mode as it works well on categorical data

In [23]:
raw_df.isnull().sum()

date          0
time          0
sensor        0
location      0
number        0
activity      0
position      0
location.1    0
dtype: int64

In [24]:
raw_df["position"] = raw_df["position"].str.lower()

In [25]:
raw_df.value_counts("position")

position
inside     171
outside      4
Name: count, dtype: int64

### Analysis 1: Datewise Total Duration for Each Location Inside and Outside

In [26]:
# Grouping by date, location, and position
grouped = raw_df.groupby(['date', 'location', 'position'])

# Calculate total duration for each location, starting on what date, and on which position
result = grouped['time'].apply(lambda x: (x.max() - x.min())).reset_index(name='total_duration')

result

,date,location,position,total_duration
0,2024-01-16,A1,inside,0 days 09:53:10
1,2024-01-16,A10,inside,0 days 10:53:00
2,2024-01-16,A11,inside,0 days 10:57:00
3,2024-01-16,A12,inside,0 days 11:00:00
4,2024-01-16,A13,inside,0 days 11:05:00
...,...,...,...,...
78,2024-01-18,A6,inside,0 days 01:14:35
79,2024-01-18,A6,outside,0 days 00:00:00
80,2024-01-18,A7,inside,0 days 02:28:33
81,2024-01-18,A8,inside,0 days 02:37:25


In [27]:
result.to_csv('task3_subtask1_output.csv', index=False)

### Analysis 2: Datewise Number of Picking and Placing Activities

In [28]:
activity_counts = raw_df.groupby(['date', 'activity']).size().unstack(fill_value=0)

In [29]:
print(activity_counts)

activity    picked  placed
date                      
2024-01-16      40      40
2024-01-17      10       9
2024-01-18      37      39


In [30]:
activity_counts.to_csv('task3_subtask2_output.csv')